# 做个小型测试  
1. 直接用vgg16的模型做seg  
2. 直接用encoder-decoder模块做seg  

## 保存decoder预训练权重

In [1]:
from keras.models import Model, load_model
from keras.layers import Input
from ae_model import encoder_vgg16, decoder1_vgg16, decoder2_vgg16, decoder3_vgg16, encoder1_vgg16
import numpy as np
import tensorflow as tf

In [ ]:
ae_model = load_model('./grid-search/checkpoints/emps_models/', compile=False)
ae_model.summary()

In [2]:
ins = Input(shape=(224, 224, 3))
encoder_model = Model(ins, encoder_vgg16(ins))
print(len(encoder_model.layers))
##34

34


In [9]:
for l1, l2 in zip(encoder_model.layers[:34], ae_model.layers[0:34]):
    l1.set_weights(l2.get_weights())

ae_weight = ae_model.get_weights()[0][1]
encoder_weight = encoder_model.get_weights()[0][1]

In [11]:
np.save('./grid-search/checkpoints/emps_models_encoder_weights_0_1.npy', encoder_weight) # for comparison
encoder_model.save('./grid-search/checkpoints/emps_models_encoder_weights.h5') # for encoder-decoder test

In [ ]:
encoder_vgg16_model = tf.keras.models.Sequential(encoder_model.layers[:-2])
encoder_vgg16_model.summary()

In [17]:
test_weight = encoder_vgg16_model.get_weights()[0][1]

In [18]:
encoder_vgg16_model.save('./grid-search/checkpoints/emps_models_encoder_vgg16_weights.h5')

### 构建vgg16 并载入保存的权重

In [3]:
load_weight_model = Model(ins, encoder_vgg16(ins))

In [4]:
conn_model = tf.keras.models.Sequential(load_weight_model.layers[:-2])

In [5]:
conn_model.load_weights('./grid-search/checkpoints/emps_models_encoder_vgg16_weights.h5')

In [6]:
conn_model_weight = conn_model.get_weights()[0][1]
compare_weight = np.load('./grid-search/checkpoints/emps_models_encoder_weights_0_1.npy')

if conn_model_weight.all() == compare_weight.all():
    print("Both weights are identical")
else: 
    print("Something wrong, weghts are different")

Both weights are identical


In [7]:
from keras.layers import Flatten, Dense

conn_model.add(Flatten())
conn_model.add(Dense(4096, activation='relu'))
conn_model.add(Dense(4096, activation='relu'))
conn_model.add(Dense(1, activation='sigmoid'))

conn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_14 (Bat  (None, 224, 224, 64)     256       
 chNormalization)                                                
                                                                 
 conv2d_15 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_15 (Bat  (None, 224, 224, 64)     256       
 chNormalization)                                                
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                        

In [13]:
import segmentation_models as sm
from keras.callbacks import TensorBoard
from datetime import datetime

logssss = './grid-search/logs/segmentation_test/pretrained_vgg16_' + datetime.now().strftime("%Y%m%d-%H%M%S") 
conn_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=[sm.metrics.iou_score])


In [9]:
x_train = np.load('./grid-search/dataset-emps/npy/spilt_80train_20test/x_train.npy')
y_train = np.load('./grid-search/dataset-emps/npy/spilt_80train_20test/y_train.npy')
x_test = np.load('./grid-search/dataset-emps/npy/spilt_80train_20test/x_test.npy')
y_test = np.load('./grid-search/dataset-emps/npy/spilt_80train_20test/y_test.npy')

In [ ]:
conn_model.fit(x_train, y_train, verbose=1, batch_size=16, validation_data=(x_test, y_test), epochs=50, shuffle=False, callbacks=[TensorBoard(log_dir=logssss)])

### 问题  
   ValueError: `logits` and `labels` must have the same shape, received ((None, 1) vs (None, 224, 224, 1)).    
VGG16最原始的模型不能拿来这么用，我的数据不是这个结构。      
Dense层出来的结构不一样.... 所以基本不能这么训练。   
